In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
HH gate curves: alpha(V), beta(V), alpha+beta, d(alpha)/dV, d(beta)/dV, k(V)
for m, h, n gates. Prints bounds and saves plots to PNG files.

Formulas match the HH code you gave:
  m_gate:
    alpha = 0.1 * vtrap(-(V + 40), 10)
    beta  = 4.0 * exp(-(V + 65)/18)
  h_gate:
    alpha = 0.07 * exp(-(V + 65)/20)
    beta  = 1.0 / (exp(-(V + 35)/10) + 1)
  n_gate:
    alpha = 0.01 * vtrap(-(V + 55), 10)
    beta  = 0.125 * exp(-(V + 65)/80)

k(V) is defined here as alpha + beta (i.e., 1/tau).
"""

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable, Tuple, Dict


# ---------- numerically stable helpers ----------

def safe_exp(x: np.ndarray) -> np.ndarray:
    # Clip to avoid overflow/underflow warnings; range is ample for HH voltages
    return np.exp(np.clip(x, -700.0, 700.0))

def vtrap(x: np.ndarray, y: float) -> np.ndarray:
    """
    Numerically stable version of x / (exp(x/y) - 1).
    Uses series expansion around x/y ~ 0: x/(e^{u}-1) ~ y*(1 - u/2 + u^2/12 - ...).
    """
    u = x / y
    small = np.abs(u) < 1e-6
    out = np.empty_like(u)
    out[~small] = x[~small] / np.expm1(u[~small])
    # 2-term series is enough for our ranges
    out[small] = y * (1.0 - 0.5 * u[small])
    return out


# ---------- HH gates (alpha, beta) ----------

def m_gate_alpha(V: np.ndarray) -> np.ndarray:
    return 0.1 * vtrap(-(V + 40.0), 10.0)

def m_gate_beta(V: np.ndarray) -> np.ndarray:
    return 4.0 * safe_exp(-(V + 65.0) / 18.0)

def h_gate_alpha(V: np.ndarray) -> np.ndarray:
    return 0.07 * safe_exp(-(V + 65.0) / 20.0)

def h_gate_beta(V: np.ndarray) -> np.ndarray:
    return 1.0 / (safe_exp(-(V + 35.0) / 10.0) + 1.0)

def n_gate_alpha(V: np.ndarray) -> np.ndarray:
    return 0.01 * vtrap(-(V + 55.0), 10.0)

def n_gate_beta(V: np.ndarray) -> np.ndarray:
    return 0.125 * safe_exp(-(V + 65.0) / 80.0)


@dataclass
class GateFuncs:
    alpha: Callable[[np.ndarray], np.ndarray]
    beta: Callable[[np.ndarray], np.ndarray]
    name: str


GATES: Dict[str, GateFuncs] = {
    "m": GateFuncs(m_gate_alpha, m_gate_beta, "m"),
    "h": GateFuncs(h_gate_alpha, h_gate_beta, "h"),
    "n": GateFuncs(n_gate_alpha, n_gate_beta, "n"),
}


# ---------- core evaluation ----------

def eval_gate_curves(
    gate: GateFuncs, V: np.ndarray
) -> Dict[str, np.ndarray]:
    a = gate.alpha(V)
    b = gate.beta(V)
    apb = a + b
    # numerical derivatives d/dV (central diff via np.gradient)
    a_prime = np.gradient(a, V)
    b_prime = np.gradient(b, V)
    k = apb  # define k(V) = alpha + beta
    return {
        "alpha": a,
        "beta": b,
        "alpha_plus_beta": apb,
        "alpha_prime": a_prime,
        "beta_prime": b_prime,
        "k": k,
    }


def print_bounds(name: str, V: np.ndarray, curves: Dict[str, np.ndarray]) -> None:
    print(f"\n==== {name}-gate over V ∈ [{V.min():.1f}, {V.max():.1f}] mV ====")
    for key, arr in curves.items():
        mn = float(np.nanmin(arr))
        mx = float(np.nanmax(arr))
        print(f"{key:>16s}: min = {mn:.6g}, max = {mx:.6g}")


# ---------- plotting (saved to files; no GUI popups) ----------

def plot_and_save(V: np.ndarray, Y: np.ndarray, title: str, ylabel: str, fname: str) -> None:
    plt.figure(figsize=(7, 4.2))
    plt.plot(V, Y)
    plt.xlabel("V (mV)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(fname, dpi=160)
    plt.close()


def save_all_plots(gate_name: str, V: np.ndarray, curves: Dict[str, np.ndarray]) -> None:
    tag = gate_name.lower()
    plot_and_save(V, curves["alpha"], f"{gate_name}-gate: alpha(V)", "alpha", f"{tag}_alpha.png")
    plot_and_save(V, curves["beta"],  f"{gate_name}-gate: beta(V)",  "beta",  f"{tag}_beta.png")
    plot_and_save(V, curves["alpha_plus_beta"], f"{gate_name}-gate: alpha+beta", "alpha+beta", f"{tag}_alpha_plus_beta.png")
    plot_and_save(V, curves["alpha_prime"], f"{gate_name}-gate: d(alpha)/dV", "d alpha / dV", f"{tag}_alpha_prime.png")
    plot_and_save(V, curves["beta_prime"], f"{gate_name}-gate: d(beta)/dV", "d beta / dV", f"{tag}_beta_prime.png")
    plot_and_save(V, curves["k"], f"{gate_name}-gate: k(V) = alpha+beta", "k", f"{tag}_k.png")


# ---------- main ----------

def main():
    # Sensible HH voltage range; adjust as needed
    V = np.linspace(-120.0, 60.0, 2001)  # mV

    for name, gate in GATES.items():
        curves = eval_gate_curves(gate, V)
        print_bounds(name, V, curves)
        save_all_plots(name, V, curves)

    print("\nSaved files:")
    print("  m_alpha.png, m_beta.png, m_alpha_plus_beta.png, m_alpha_prime.png, m_beta_prime.png, m_k.png")
    print("  h_alpha.png, h_beta.png, h_alpha_plus_beta.png, h_alpha_prime.png, h_beta_prime.png, h_k.png")
    print("  n_alpha.png, n_beta.png, n_alpha_plus_beta.png, n_alpha_prime.png, n_beta_prime.png, n_k.png")

if __name__ == "__main__":
    main()



==== m-gate over V ∈ [-120.0, 60.0] mV ====
           alpha: min = 0.0026846, max = 10.0005
            beta: min = 0.0038559, max = 84.9319
 alpha_plus_beta: min = 1.99428, max = 84.9346
     alpha_prime: min = 0.000235902, max = 0.099959
      beta_prime: min = -4.70666, max = -0.000214753
               k: min = 1.99428, max = 84.9346

==== h-gate over V ∈ [-120.0, 60.0] mV ====
           alpha: min = 0.000135132, max = 1.09498
            beta: min = 0.000203427, max = 0.999925
 alpha_plus_beta: min = 0.116518, max = 1.09519
     alpha_prime: min = -0.0546262, max = -6.77181e-06
      beta_prime: min = 7.51784e-06, max = 0.0249997
               k: min = 0.116518, max = 1.09519

==== n-gate over V ∈ [-120.0, 60.0] mV ====
           alpha: min = 0.000978707, max = 1.15001
            beta: min = 0.0262014, max = 0.248592
 alpha_plus_beta: min = 0.172642, max = 1.17621
     alpha_prime: min = 8.32684e-05, max = 0.00999893
      beta_prime: min = -0.00310566, max = -0.000327702
  